# Solution: Image Data Augmentation

[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edgeimpulse/workshop-arduino-tinyml-roshambo/blob/main/02-data-augmentation/ei-image-augmentation.ipynb)

This is a script for creating an augmented dataset for images. It transforms input images to create a series of augmented samples that are then uploaded to your Edge Impulse project.

Upload your dataset as *dataset.zip* to */content/*. All of your samples should have the filename format `<label>.<unique-id>.png` (.jpg is also acceptable).

Create a new project on [Edge Impulse](https://edgeimpulse.com/). Go to the dashboard of that project and copy your API key. Paste that key into the string value for `EI_API_KEY`. The final cell of this script will automatically upload the augmented dataset to your Edge Impulse project.

Press **shift + enter** to execute each cell.

The original images along with their transforms will be saved in the output directory. Each output file will be the original filename appended with "_{num}" where {num} is some incrementing value based on the total number of transforms performed per image.

For example, if you have a file named `alpha.0.png`, it will become `alpha.0_0.png`. The first transform will be `alpha.0_1.png`, the second transform will be `alpha.0_2.png` and so on.

Author: EdgeImpulse, Inc.<br>
Date: January 5, 2023<br>
License: [Apache-2.0](apache.org/licenses/LICENSE-2.0)<br>

In [ ]:
### Update Node.js to the latest stable version
!npm cache clean -f
!npm install -g n
!n 16.18.1

npm warn using --force Recommended protections disabled.
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠙⠹⠸⠼⠴⠦⠧⠇
added 1 package in 2s
⠇  installing : node-v16.18.1
       mkdir : /usr/local/n/versions/node/16.18.1
       fetch : https://nodejs.org/dist/v16.18.1/node-v16.18.1-linux-x64.tar.xz
######################################################################## 100.0%
     copying : node/16.18.1
   installed : v16.18.1 (with npm 8.19.2)

Note: the node command changed location and the old location may be remembered in your current shell.
         old : /tools/node/bin/node
         new : /usr/local/bin/node
If "node --version" shows the old version then start a new shell, or reset the location hash with:
hash -r  (for bash, zsh, ash, dash, and ksh)
rehash   (for csh and tcsh)


In [ ]:
### Install required packages and tools
!npm install -g --unsafe-perm edge-impulse-cli

npm WARN EBADENGINE Unsupported engine {
npm WARN EBADENGINE   package: 'npm-registry-fetch@18.0.2',
npm WARN EBADENGINE   required: { node: '^18.17.0 || >=20.5.0' },
npm WARN EBADENGINE   current: { node: 'v16.18.1', npm: '8.19.2' }
npm WARN EBADENGINE }
npm WARN EBADENGINE Unsupported engine {
npm WARN EBADENGINE   package: 'tar@7.4.3',
npm WARN EBADENGINE   required: { node: '>=18' },
npm WARN EBADENGINE   current: { node: 'v16.18.1', npm: '8.19.2' }
npm WARN EBADENGINE }
npm WARN EBADENGINE Unsupported engine {
npm WARN EBADENGINE   package: '@aws-sdk/util-locate-window@3.893.0',
npm WARN EBADENGINE   required: { node: '>=18.0.0' },
npm WARN EBADENGINE   current: { node: 'v16.18.1', npm: '8.19.2' }
npm WARN EBADENGINE }
npm WARN EBADENGINE Unsupported engine {
npm WARN EBADENGINE   package: '@npmcli/redact@3.2.2',
npm WARN EBADENGINE   required: { node: '^18.17.0 || >=20.5.0' },
npm WARN EBADENGINE   current: { node: 'v16.18.1', npm: '8.19.2' }
npm WARN EBADENGINE }
npm WARN EBADEN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import shutil
import PIL

import skimage.transform
import skimage.util

In [ ]:
### Settings

# Copy API here: Edge Impulse > your_project > Dashboard > Keys
EI_API_KEY = "ei_f13e687849ce19baf7010433260e52b2c74e1e7e5f1fd51e4f756ec357d3aeae"

# Path information
HOME_PATH = "/content"                  # Location of the working directory
DATASET_ZIP = "/content/datasheet.zip"    # Name of the .zip file containing your original dataset
DATASET_PATH = "/content/datasheet"       # Upload your .csv samples to this directory
OUT_PATH = "/content/out"               # Where output files go (will be deleted and recreated)
OUT_ZIP = "/content/out-augmented.zip"  # Where to store the zipped output files

# How to split the dataset
TEST_RATIO = 0.2      # 20% reserved for test set, rest is for training

# File format to use for new dataset
IMG_EXT = ".png"

# Max batch size for uploading to Edge Impulse
MAX_UPLOAD_BATCH_SIZE = 100

# You are welcome to change the seed to get different augmentation effects
SEED = 42
random.seed(SEED)

In [ ]:
### Use this to unzip the dataset folder
# !rm -rf /content/dataset

In [ ]:
### Unzip files to dataset directory
%cd {HOME_PATH}
!mkdir {DATASET_PATH}
!unzip -q -d {DATASET_PATH} {DATASET_ZIP}

/content


## Transform Functions

In [ ]:
### Example: Function to create 3 new flipped images of the input
def create_flipped(img):

  # Create a list of flipped images
  flipped = []
  flipped.append(np.fliplr(img))
  flipped.append(np.flipud(img))
  flipped.append(np.flipud(np.fliplr(img)))

  return flipped

In [ ]:
### Function to create new rotated images of the input
def create_rotated(img, rotations):

  # Create list of rotated images (keep 8-bit values)
  rotated = []
  for rot in rotations:
    img_rot = skimage.transform.rotate(img, angle=rot, mode='edge', preserve_range=True)
    img_rot = img_rot.astype(np.uint8)
    rotated.append(img_rot)

  return rotated

In [ ]:
def create_random_zooms(img, scale_factor, num_crops):

  # Get height and width of original image
  height = img.shape[0]
  width = img.shape[1]

  # Create scaled images (e.g. make the image bigger) and keep 8-bit values
  img_scaled = skimage.transform.rescale(img,
                                        scale=scale_factor,
                                        anti_aliasing=True,
                                        channel_axis=-1) # Changed multichannel=True to channel_axis=-1
  img_scaled = img_scaled.astype(np.uint8)

  # Get height and width of scaled image
  s_h = img_scaled.shape[0]
  s_w = img_scaled.shape[1]

  # Create list of random zooms
  zooms = []
  for i in range(num_crops):

    # Randomly choose start of crop point
    crop_y = round(random.random() * (s_h - height))
    crop_x = round(random.random() * (s_w - width))

    # Crop scaled image
    zoom = img_scaled[crop_y:(crop_y + height), crop_x:(crop_x + width), :]

    # Append zoomed image to list
    zooms.append(zoom)

  return zooms

In [ ]:
### Function to create a random set of translated images (no more than 1/4 of width or height away)
def create_random_translations(img, num_translations):

  # Get height and width of original image
  height = img.shape[0]
  width = img.shape[1]

  # Create list of random translations
  translations = []
  for i in range(num_translations):

    # Choose random amount to translate (up to 1/4 image width, height) in either direction
    tr_y = round((0.5 - random.random()) * (height / 2))
    tr_x = round((0.5 - random.random()) * (width / 2))

    # Perform translation to create new image
    translation = skimage.transform.AffineTransform(translation=(tr_y, tr_x))
    img_tr = skimage.transform.warp(img, translation, mode='edge', preserve_range=True)
    img_tr = img_tr.astype(np.uint8)

    # Append translated image to list
    translations.append(img_tr)

  return translations

In [ ]:
### Function to add random noise to images
def create_noisy(img, types, seed=None):

  # Add noise of different types
  noisy_imgs = []
  for t in types:
    # Removed 'seed=seed' as it was causing a ValueError for some noise modes
    noise = skimage.util.random_noise(img, mode=t)
    noise = (noise * 255).astype(np.uint8)
    noisy_imgs.append(noise)

  return noisy_imgs

## Perform Transforms

In [ ]:
### Delete output directory (if it exists) and recreate it
if os.path.exists(OUT_PATH):
  shutil.rmtree(OUT_PATH)
os.makedirs(OUT_PATH)

In [ ]:
### Function to open image and create a list of new transforms
def create_transforms(file_path):

  # Open the image
  img = PIL.Image.open(file_path)

  # Convert the image to a Numpy array (keep all color channels)
  img_array = np.asarray(img)

  # Add original image to front of list
  img_tfs = []
  img_tfs.append([img_array])

  # Perform transforms (call your functions)
  img_tfs.append(create_flipped(img_array))
  img_tfs.append(create_flipped(img_array))
  img_tfs.append(create_rotated(img_array, [45, 90, 135]))
  img_tfs.append(create_random_zooms(img_array, 1.3, 2))
  img_tfs.append(create_random_translations(img_array, 2))
  img_tfs.append(create_noisy(img_array, ['gaussian', 's&p'], SEED))

  # Flatten list of lists (to create one long list of images)
  img_tfs = [img for img_list in img_tfs for img in img_list]

  return img_tfs

In [ ]:
### Load all images, create transforms, and save in output directory

# Go through each file in the unzipped directory
for filename in os.listdir(DATASET_PATH):

  # Skip the Jupyter Notebook checkpoints folder that sometimes gets added
  if filename == ".ipynb_checkpoints":
    continue

  # Parse the filename into label and unique ID
  file_root = os.path.splitext(filename)[0]
  label = file_root.split('.')[0]
  uid = '.'.join(file_root.split('.')[1:])

  # Do all transforms for that one image
  file_path = os.path.join(DATASET_PATH, filename)
  img_tfs = create_transforms(file_path)

  # Save images to new files in output directory
  for i, img in enumerate(img_tfs):

    # Create a Pillow image from the Numpy array
    img_pil = PIL.Image.fromarray(img)

    # Construct filename (<original>_<transform_num>.<EXT>)
    out_file_path = os.path.join(OUT_PATH, label + "." + uid + "_" + str(i) + IMG_EXT)

    # Convert Numpy array to image and save as a file
    img_pil = PIL.Image.fromarray(img)
    img_pil.save(out_file_path)

In [ ]:
### Zip our new dataset (use '!' to call Linux commands)
%cd {OUT_PATH}
!zip -FS -r -q {OUT_ZIP} *
%cd {HOME_PATH}

/content/out
/content


## Upload Dataset to Edge Impulse

In [ ]:
### Shuffle/split the filenames and create lists of the full paths

# Create list of files for one category
paths = []
for filename in os.listdir(OUT_PATH):
  paths.append(os.path.join(OUT_PATH, filename))

# Shuffle and divide into test and training sets
random.shuffle(paths)
num_test_samples = int(TEST_RATIO * len(paths))
test_paths = paths[:num_test_samples]
train_paths = paths[num_test_samples:]

In [ ]:
### Upload test set to Edge Impulse in mini batches
for first in range(0, len(test_paths), MAX_UPLOAD_BATCH_SIZE):

  # Construct one long string with all the paths of the mini batch
  test_mini_batch = test_paths[first:(first + MAX_UPLOAD_BATCH_SIZE)]
  print(f"Uploading {len(test_mini_batch)} files. Number {first} to " \
        f"{first + MAX_UPLOAD_BATCH_SIZE} out of a total of {len(test_paths)}.")
  test_mini_batch = ['"' + s + '"' for s in test_mini_batch]
  test_mini_batch = ' '.join(test_mini_batch)

  # Upload to Edge Impulse
  !edge-impulse-uploader \
  --category testing \
  --api-key {EI_API_KEY} \
  --silent \
  {test_mini_batch}

Uploading 100 files. Number 0 to 100 out of a total of 1366.
[  1/100] Uploading /content/out/NATE.53ca364eab50_1.png OK (477 ms)
[  2/100] Uploading /content/out/NATE.4d2223b7f947_12.png OK (479 ms)
[  3/100] Uploading /content/out/NATE.49a82a953a28_7.png OK (480 ms)
[  4/100] Uploading /content/out/BACKGROUND.875eaa37325b_10.png OK (496 ms)
[  5/100] Uploading /content/out/UNKNOWN.4eaea06fa3c0_7.png OK (506 ms)
[  6/100] Uploading /content/out/BACKGROUND.153b77b56465_10.png OK (506 ms)
[  7/100] Uploading /content/out/NATE.7f4bf9af32d7_3.png OK (510 ms)
[  8/100] Uploading /content/out/UNKNOWN.cff7584e7efa_1.png OK (515 ms)
[  9/100] Uploading /content/out/NATE.f2995a7ac111_13.png OK (519 ms)
[ 10/100] Uploading /content/out/BACKGROUND.03594765ced2_14.png OK (522 ms)
[ 11/100] Uploading /content/out/NATE.269dc3080c49_11.png OK (524 ms)
[ 12/100] Uploading /content/out/BACKGROUND.5545c891df92_1.png OK (529 ms)
[ 13/100] Uploading /content/out/NATE.156e260a8a11_7.png OK (534 ms)
[ 14/1

In [ ]:
### Upload training set to Edge Impulse in mini batches
for first in range(0, len(train_paths), MAX_UPLOAD_BATCH_SIZE):

  # Construct one long string with all the paths of the mini batch
  train_mini_batch = train_paths[first:(first + MAX_UPLOAD_BATCH_SIZE)]
  print(f"Uploading {len(train_mini_batch)} files. Number {first} to " \
        f"{first + MAX_UPLOAD_BATCH_SIZE} out of a total of {len(train_paths)}.")
  train_mini_batch = ['"' + s + '"' for s in train_mini_batch]
  train_mini_batch = ' '.join(train_mini_batch)

  # Upload to Edge Impulse
  !edge-impulse-uploader \
  --category training \
  --api-key {EI_API_KEY} \
  --silent \
  {train_mini_batch}

Streaming output truncated to the last 5000 lines.
[ 16/100] Uploading /content/out/UNKNOWN.e34715ab85f6_4.png OK (444 ms)
[ 17/100] Uploading /content/out/NATE.663809158ee3_6.png OK (451 ms)
[ 18/100] Uploading /content/out/BACKGROUND.558ef0277a4c_14.png OK (454 ms)
[ 19/100] Uploading /content/out/NATE.7dcb45a76fbe_12.png OK (457 ms)
[ 20/100] Failed to upload /content/out/UNKNOWN.7e1369d01f9d_5.png An item with this hash already exists (ids: 2459031704)
[ 21/100] Failed to upload /content/out/BACKGROUND.999c6b06c5c8_3.png An item with this hash already exists (ids: 2459032385)
[ 22/100] Uploading /content/out/UNKNOWN.68f365b0428a_6.png OK (506 ms)
[ 23/100] Uploading /content/out/NATE.5eb7d49441b3_15.png OK (186 ms)
[ 24/100] Failed to upload /content/out/BACKGROUND.38bae6a63d88_10.png An item with this hash already exists (ids: 2459031389, 2459031391)
[ 25/100] Uploading /content/out/UNKNOWN.2fbee50375a7_8.png OK (184 ms)
[ 26/100] Uploading /content/out/UNKNOWN.390e5e916670_4.png 